## SET UP

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [19]:
!git clone https://github.com/LauraMJanssen/esrgan.git
%cd esrgan/
!pip install -r requirements.txt

Cloning into 'esrgan'...
remote: Enumerating objects: 44, done.
remote: Total 44 (delta 0), reused 0 (delta 0), pack-reused 44
Unpacking objects: 100% (44/44), done.
/content/esrgan
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
!rm -r /content/esrgan
%cd /content/

/content


## For transfer learning:

Upload necessary files: checkpoint file & 2 weight files or esrgan_inference.zip (=pre-trained model)

In [20]:
! mkdir -p checkpoints/esrgan

For esrgan_inference:

In [21]:
! unzip "/content/esrgan/pretrainedModel/esrgan_inference.zip" -d "/content/esrgan/pretrainedModel/"

Archive:  /content/esrgan/pretrainedModel/esrgan_inference.zip
   creating: /content/esrgan/pretrainedModel/esrgan_inference/
  inflating: /content/esrgan/pretrainedModel/esrgan_inference/checkpoint  
  inflating: /content/esrgan/pretrainedModel/esrgan_inference/ckpt-581.data-00000-of-00002  
  inflating: /content/esrgan/pretrainedModel/esrgan_inference/ckpt-581.data-00001-of-00002  
  inflating: /content/esrgan/pretrainedModel/esrgan_inference/ckpt-581.index  


In [22]:
! mv /content/esrgan/pretrainedModel/esrgan_inference/* checkpoints/esrgan

For other weights etc: adapt names! 

## Data Processing \\
-> needs to be in '.tfrecod' format

In [ ]:
! rm /content/esrgan/data/DIV2K800_sub_bin.tfrecord
! python data/convert_train_tfrecord.py --hr_dataset_path='/content/gdrive/MyDrive/MA/IVUSImages_PNG_Train2' --lr_dataset_path='/content/gdrive/MyDrive/MA/IVUS_LowRes_Scale4_Train' --output_path="./data/DIV2K800_sub_bin.tfrecord" --is_binary=True

rm: cannot remove '/content/esrgan/data/DIV2K800_sub_bin.tfrecord': No such file or directory
2022-08-27 11:26:33.991041: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-08-27 11:26:33.991231: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-08-27 11:26:33.991252: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
I0827 11:26:34.801126 140589794490240 convert_train_tfrecord.py:62] Loading /content/gdrive/MyDrive/MA/IVUSImages_P

In [ ]:
! rm /content/esrgan/data/DIV2K800_sub_bin_valid.tfrecord
! python data/convert_train_tfrecord.py --hr_dataset_path='/content/gdrive/MyDrive/MA/IVUSImages_PNG_Valid2' --lr_dataset_path='/content/gdrive/MyDrive/MA/IVUS_LowRes_Scale4_Valid' --output_path="./data/DIV2K800_sub_bin_valid.tfrecord" --is_binary=True

## Train

In [ ]:
! python train_esrgan.py --cfg_path="./configs/esrgan.yaml" --gpu=1

In [ ]:
!zip -r /content/ESRGAN_model.zip /content/esrgan-tf2/checkpoints/esrgan

  adding: content/esrgan-tf2/checkpoints/esrgan/ (stored 0%)
  adding: content/esrgan-tf2/checkpoints/esrgan/ckpt-5.index (deflated 83%)
  adding: content/esrgan-tf2/checkpoints/esrgan/ckpt-6.data-00000-of-00001 (deflated 8%)
  adding: content/esrgan-tf2/checkpoints/esrgan/ckpt-6.index (deflated 83%)
  adding: content/esrgan-tf2/checkpoints/esrgan/checkpoint (deflated 61%)
  adding: content/esrgan-tf2/checkpoints/esrgan/ckpt-5.data-00000-of-00001 (deflated 8%)


In [ ]:
from google.colab import files
files.download("/content/ESRGAN_model.zip")

## TEST

(1) Define quality metric functions

In [ ]:
from skimage.metrics import structural_similarity as ssim

# define a function for peak signal-to-noise ratio (PSNR)
def psnr(target, ref):
         
    # assume RGB image
    target_data = target.astype(float)
    ref_data = ref.astype(float)

    diff = ref_data - target_data
    diff = diff.flatten('C')

    rmse = math.sqrt(np.mean(diff ** 2.))

    return 20 * math.log10(255. / rmse)

# define function for mean squared error (MSE)
def mse(target, ref):
    # the MSE between the two images is the sum of the squared difference between the two images
    err = np.sum((target.astype('float') - ref.astype('float')) ** 2)
    err /= float(target.shape[0] * target.shape[1])
    
    return err

# define function that combines all three image quality metrics
def compare_images(target, ref):
    scores = []
    scores.append(psnr(target, ref))
    scores.append(mse(target, ref))
    scores.append(ssim(target, ref, multichannel =True))
    
    return scores

(2) Execute model on specific image

In [7]:
from IPython.display import Image
!python test.py --cfg_path="./configs/esrgan.yaml" --img_path="/content/gdrive/MyDrive/MA/IVUSImages_PNG_Valid2/001.png"

2022-08-26 23:55:25.246363: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.7/dist-packages/cv2/../../lib64:/usr/lib64-nvidia
2022-08-26 23:55:25.246449: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.7/dist-packages/cv2/../../lib64:/usr/lib64-nvidia
2022-08-26 23:55:25.246464: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
I0826 23:55:26.170805 140266753300352 utils.py:30] Detect 1 Physical GPUs, 1 Logical GPUs.
[*] load

(3) Print results
(make sure to adapt ref and degraded parts according to image read above)

In [ ]:
import math
import numpy as np
import cv2

ref = cv2.imread('/content/gdrive/MyDrive/MA/IVUSImages_PNG_Valid2/001.png')
degraded = cv2.imread('/content/gdrive/MyDrive/MA/IVUS_LowRes_Scale4_Valid/001.png')
output = cv2.imread('/content/result.png')
  
# image quality calculations
scores = []
h, w, _ = degraded.shape
degraded2 = cv2.resize(degraded, (4*w, 4*h))
scores.append(compare_images(degraded2, ref))

scores.append(compare_images(output, ref))

  
# print all scores for all images
print('Degraded Image: \nPSNR: {}\nMSE: {}\nSSIM: {}\n'.format(scores[0][0], scores[0][1], scores[0][2]))
print('Reconstructed Image: \nPSNR: {}\nMSE: {}\nSSIM: {}\n'.format(scores[1][0], scores[1][1], scores[1][2]))

Degraded Image: 
PSNR: 29.40398264707939
MSE: 223.77076721191406
SSIM: 0.8560081594432712

Reconstructed Image: 
PSNR: 24.284026710372892
MSE: 727.4429588317871
SSIM: 0.7063703437527753

